# import

In [6]:
## import functions from .py utils file: 
from Utilities.preprocessing import *
import os

# paths

In [3]:
InputPath = r"Z:\projects\sequence_squad\revision_data\lars_recordings\bpod_data\\"


# #Set paths 
# Animal_ID = ['EJT255']
# #'EJT149','EJT162','EJT178',

# InputPath = (r'Z:\projects\sequence_squad\data\raw_bpod')
# CameraPath = (r'Z:\projects\sequence_squad\data\video_data\raw_flycap')

# OutputPath = (r'Z:\projects\sequence_squad\data\processed_bpod_data')

In [13]:
main_paths= []
for mouse in os.listdir(InputPath):
    if not mouse == 'FakeSubject':
        os.listdir(os.path.join(InputPath,mouse))
        main_paths+=[os.path.join(InputPath,mouse,'Sequence_Automated','Session Data')]

In [ ]:
os.listdir()

['Sequence_Automated_CurrentTrainingLevel.mat',
 'SP156_Sequence_Automated_20240702_141249.fig',
 'SP156_Sequence_Automated_20240702_141249.mat',
 'SP156_Sequence_Automated_20240702_143224.fig',
 'SP156_Sequence_Automated_20240702_143224.mat',
 'SP156_Sequence_Automated_20240702_144056.fig',
 'SP156_Sequence_Automated_20240702_144056.mat',
 'SP156_Sequence_Automated_20240703_142450.fig',
 'SP156_Sequence_Automated_20240703_142450.mat',
 'SP156_Sequence_Automated_20240703_143420.fig',
 'SP156_Sequence_Automated_20240703_143420.mat',
 'SP156_Sequence_Automated_20240704_143008.fig',
 'SP156_Sequence_Automated_20240704_143008.mat',
 'SP156_Sequence_Automated_20240705_140630.fig',
 'SP156_Sequence_Automated_20240705_140630.mat',
 'SP156_Sequence_Automated_20240708_153728.fig',
 'SP156_Sequence_Automated_20240708_153728.mat',
 'SP156_Sequence_Automated_20240709_112852.fig',
 'SP156_Sequence_Automated_20240709_112852.mat',
 'SP156_Sequence_Automated_20240710_113635.fig',
 'SP156_Sequence_Auto

In [1]:
#### MAIN ####
for Animal_index, CurrentAnimal in enumerate(Animal_ID):
    print ('Processing data for: ' + CurrentAnimal)
    InputPathCurrent = os.path.join(InputPath,CurrentAnimal) + '\\' 
    
    #Load Behavioural data:
    Behav_data, Sessions,Path,FileDates = Import_Bpod_DataFiles(InputPathCurrent)
    
    Processed = ('')
    Skipped = ('')
 

    for Session in range(0,2):
#     for Session in range(17,Sessions):
        print(Session)

        filedate = FileDates[Session] + '_' + str(Behav_data[Session]['__header__'])[-25:-22]

        if Session < 10:
            Save_path = (OutputPath + '\\'+ CurrentAnimal + '\Preprocessed' + '\\0' + str(Session) + '_' + filedate)
        else:
            Save_path = (OutputPath + '\\' + CurrentAnimal + '\Preprocessed' + '\\' + str(Session) + '_' + filedate)

        # see if dir exists already then use bool logic to detemrine if it needs to be processed or not
        if not os.path.isdir(Save_path):
            os.makedirs(Save_path)
            Process = True
        else:
            if  Replace == True:
                Process = True
            else:
                Process = False

        if  Process == True:

            #Convert to python friendly format:
            convert_nested_structs(Behav_data[Session])

        #         # Extract GUI info
            Trial_settings = todict(Behav_data[Session]['SessionData']['TrialSettings'][0])
            FinalRewardAmount = []
            for item in Behav_data[Session]['SessionData']['SessionVariables']['TLevel']:
                TLevel = item
                FinalRewardAmount = FinalRewardAmount + [Behav_data[Session]['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][4]]

            # save out training levels on their own
            filename = 'Preprocessed_TrainingLevels' 
            with open(Save_path + '\\'+ filename, 'wb') as fp:
                pickle.dump(Behav_data[Session]['SessionData']['SessionVariables']['TLevel'], fp)

            # save out led intensites and reward amounts on their own:
            LED_Intensities = pd.DataFrame({'Port1':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port1'],
                                            'Port2':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port2'],
                                            'Port3':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port3'],
                                            'Port4':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port4'],
                                            'Port5':Behav_data[Session]['SessionData']['SessionVariables']['LEDIntensitys']['port5']})
            LED_Intensities.to_csv(Save_path + '/Preprocessed_LED_Intensities.csv')
            RewardAmounts = pd.DataFrame({'Port1':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port1'],
                                           'Port2':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port2'],
                                           'Port3':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port3'],
                                           'Port4':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port4'],
                                           'Port5':Behav_data[Session]['SessionData']['SessionVariables']['RewardAmount']['port5']})
            RewardAmounts.to_csv(Save_path + '/Preprocessed_RewardAmounts.csv')


            #Extract PortIn times for each port and check for errors (inside this function):
            All_PortIn_Times,All_PortOut_Times,All_Port_references = extract_poke_times(Behav_data[Session])

            
            #remove nans (times when part [in or out poke] of the event was dropped for some reason by bpod)
            All_PortIn_Times_fixed,All_PortOut_Times_fixed ,All_Port_references_fixed = remove_dropped_in_events(All_PortIn_Times,All_PortOut_Times,All_Port_references)
            
            # Resort these in time:
            All_PortIn_Times_sorted,All_PortOut_Times_sorted,All_Port_references_sorted = time_sort(All_PortIn_Times,All_PortOut_Times,All_Port_references)

            #extract reward times:
            Reward_ts = extract_reward_times(Behav_data[Session])
            
            # fix bullshit error that appeared
            All_PortIn_Times_sorted = All_PortIn_Times_sorted[0]
            All_Port_references_sorted = All_Port_references_sorted[0]
            All_PortOut_Times_sorted = All_PortOut_Times_sorted[0]

            # find reward inds and align rewarded ts to poke events:
            Rewarded_event_inds = find_reward_inds(All_PortIn_Times_sorted,All_Port_references_sorted,Reward_ts)
            Reward_ts = np.asarray(Reward_ts)
            Reward_ts = Reward_ts[np.logical_not(np.isnan(Reward_ts))]
            Reward_ts = list(Reward_ts)
            Reward_ts_aligned = align_trigger_to_index(Reward_ts,Rewarded_event_inds,All_Port_references_sorted)

            #extract trial start time stamps
            Trial_start_ts = extract_trial_timestamps(Behav_data[Session])

            #extract trial end times:
            Trial_end_ts = extract_trial_end_times(Behav_data[Session])

            #determine trial IDs
            trial_id = determine_trial_id(All_PortIn_Times_sorted,Trial_end_ts)

            # align trial start and end times to poke events
            trialstart_index = find_trialstart_index(trial_id)
            trial_start_ts_aligned = align_trial_start_end_timestamps(trial_id,trialstart_index,Trial_start_ts)
            trial_end_ts_aligned = align_trial_start_end_timestamps(trial_id,trialstart_index,Trial_end_ts)

        
            #determine LED and reward states for each trial and align them to trials:
            IntermediateRewards = []
            LED_intensities = []
            for item in Behav_data[Session]['SessionData']['SessionVariables']['TLevel']:
                TLevel = item
                IntermediateRewards = IntermediateRewards + [list(Behav_data[Session]['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][0:4])]
                LED_intensities = LED_intensities + [list(Behav_data[Session]['SessionData']['SessionVariables']['TrainingLevels'][TLevel-1][6:10])]
            aligned_LED_intensities = align_trial_start_end_timestamps(trial_id,trialstart_index,LED_intensities)
            aligned_IntermediateRewards = align_trial_start_end_timestamps(trial_id,trialstart_index,IntermediateRewards)

            #Search for timestamps for given animal and session 
            TimeStampsExist,Camera_started_after, TimeStampPaths = FindEphysTimestamps(filedate,CameraPath,CurrentAnimal)
            
            
            ## shitty fix for 24/01/23
            TimeStampsExist = False
            for index, TimeStampPath in enumerate(TimeStampPaths):
                print(TimeStampPath)
                video_name = process_camera_data2(TimeStampPath,Trial_start_ts,trial_id,trialstart_index,All_Port_references_sorted,Trial_end_ts)
            
            if TimeStampsExist:
                #process camera data:
                for index, TimeStampPath in enumerate(TimeStampPaths):
                    print(TimeStampPath)
                    if index == 0:
                        Trial_start_camera_inds, Trial_start_Camera_Ts_aligned, Trial_end_Camera_Ts_aligned, First_poke_Camera_Ts_aligned,video_name  = process_camera_data(TimeStampPath,Trial_start_ts,trial_id,trialstart_index,All_Port_references_sorted,Trial_end_ts)
                    elif index == 1:
                        out1,out2,out3,out4,out5  = process_camera_data(TimeStampPath,Trial_start_ts,trial_id,trialstart_index,All_Port_references_sorted,Trial_end_ts)
                        Trial_start_camera_inds = [Trial_start_camera_inds] + [out1]
                        Trial_start_Camera_Ts_aligned = [Trial_start_Camera_Ts_aligned] + [out2]
                        Trial_end_Camera_Ts_aligned = [Trial_end_Camera_Ts_aligned] + [out3]
                        First_poke_Camera_Ts_aligned = [First_poke_Camera_Ts_aligned] + [out4]
                        video_name = [video_name] + [out5]
                    else:
                        out1,out2,out3,out4,out5  = process_camera_data(TimeStampPath,Trial_start_ts,trial_id,trialstart_index,All_Port_references_sorted,Trial_end_ts)
                        Trial_start_camera_inds.append(out1)
                        Trial_start_Camera_Ts_aligned.append(out2)
                        Trial_end_Camera_Ts_aligned.append(out3) 
                        First_poke_Camera_Ts_aligned.append(out4)
                        video_name.append(out5)
                        
            else:
                Trial_start_Camera_Ts_aligned = ['Cameras Corrupted  no video data'] * len(trial_id)
                Trial_end_Camera_Ts_aligned = ['Cameras Corrupted  no video data'] * len(trial_id)
                First_poke_Camera_Ts_aligned = ['Cameras Corrupted  no video data'] * len(trial_id)

            ## align Training level for each trial:
            Training_Levels = align_opto_trials_to_dataframe(trial_id,Behav_data[Session]['SessionData']['SessionVariables']['TLevel'])

            # make portin dataframe:
            PortIn_df = pd.DataFrame(
                {'Trial_id' : trial_id,
                 'Trial_Start' : trial_start_ts_aligned,
                 'Port': All_Port_references_sorted,
                 'PokeIn_Time': All_PortIn_Times_sorted,
                 'PokeOut_Time': All_PortOut_Times_sorted,
                 'Reward_Times': Reward_ts_aligned,
                 'Trial_End' : trial_end_ts_aligned,
                 'Port 2,3,4,5 LED intensities': aligned_LED_intensities,
                 'Port 1,2,3,4 RewardAmount':aligned_IntermediateRewards,
                 'BACK_Trial_Start_Camera_Time' : Trial_start_Camera_Ts_aligned[index],
                 'BACK_Trial_End_Camera_Time' : Trial_end_Camera_Ts_aligned[index],
                 'BACK_First_poke_Camera_Time' : First_poke_Camera_Ts_aligned[index],
                 'SIDE_Trial_Start_Camera_Time' : Trial_start_Camera_Ts_aligned[index],
                 'SIDE_Trial_End_Camera_Time' : Trial_end_Camera_Ts_aligned[index],
                 'SIDE_First_poke_Camera_Time' : First_poke_Camera_Ts_aligned[index],

                 'BOTTOM_Trial_Start_Camera_Time' : Trial_start_Camera_Ts_aligned[index],
                 'BOTTOM_Trial_End_Camera_Time': Trial_end_Camera_Ts_aligned[index],
                 'BOTTOM_First_poke_Camera_Time' : First_poke_Camera_Ts_aligned[index],

                'TrainingLevel': Training_Levels})

#             for index, item in enumerate(video_name):
#                 if 'back' in item[0]:
#                     PortIn_df['BACK_Trial_Start_Camera_Time'] = Trial_start_Camera_Ts_aligned[index]
#                     PortIn_df['BACK_Trial_End_Camera_Time'] = Trial_end_Camera_Ts_aligned[index]
#                     PortIn_df['BACK_First_poke_Camera_Time'] = First_poke_Camera_Ts_aligned[index]
#                 if 'side' in item[0]:
#                     PortIn_df['SIDE_Trial_Start_Camera_Time'] = Trial_start_Camera_Ts_aligned[index]
#                     PortIn_df['SIDE_Trial_End_Camera_Time'] = Trial_end_Camera_Ts_aligned[index]
#                     PortIn_df['SIDE_First_poke_Camera_Time'] = First_poke_Camera_Ts_aligned[index]
#                 if 'bottom' in item[0]:
#                     PortIn_df['BOTTOM_Trial_Start_Camera_Time'] = Trial_start_Camera_Ts_aligned[index]
#                     PortIn_df['BOTTOM_Trial_End_Camera_Time'] = Trial_end_Camera_Ts_aligned[index]
#                     PortIn_df['BOTTOM_First_poke_Camera_Time'] = First_poke_Camera_Ts_aligned[index]

            #Save Data
            PortIn_df.to_csv(Save_path +'/PreProcessed_RawPokeData.csv')
                        
         
            

            PortIn_df.to_csv(Save_path +'/PreProcessed_RawPokeData.csv')


<>:20: SyntaxWarning: invalid escape sequence '\P'
<>:22: SyntaxWarning: invalid escape sequence '\P'
<>:20: SyntaxWarning: invalid escape sequence '\P'
<>:22: SyntaxWarning: invalid escape sequence '\P'
C:\Users\Emmett Thompson\AppData\Local\Temp\ipykernel_33668\364952113.py:20: SyntaxWarning: invalid escape sequence '\P'
  Save_path = (OutputPath + '\\'+ CurrentAnimal + '\Preprocessed' + '\\0' + str(Session) + '_' + filedate)
C:\Users\Emmett Thompson\AppData\Local\Temp\ipykernel_33668\364952113.py:22: SyntaxWarning: invalid escape sequence '\P'
  Save_path = (OutputPath + '\\' + CurrentAnimal + '\Preprocessed' + '\\' + str(Session) + '_' + filedate)
C:\Users\Emmett Thompson\AppData\Local\Temp\ipykernel_33668\364952113.py:20: SyntaxWarning: invalid escape sequence '\P'
  Save_path = (OutputPath + '\\'+ CurrentAnimal + '\Preprocessed' + '\\0' + str(Session) + '_' + filedate)
C:\Users\Emmett Thompson\AppData\Local\Temp\ipykernel_33668\364952113.py:22: SyntaxWarning: invalid escape seque

NameError: name 'Animal_ID' is not defined